In [2]:
import pandas as pd
import ee
import numpy as np
import geopandas as gpd
from shapely.geometry import MultiPoint
from config import RECTANGLE, DATES
#ee.Authenticate()

# Initialize Earth Engine
ee.Initialize()

# Extraer datos

This product is the concentration of the photosynthetic pigment (chlorophyll-a) in phytoplankton in the sea surface layer.

This is an ongoing dataset with a latency of 3-4 days.

GCOM-C conducts long-term and continuous global observation and data collection to elucidate the mechanism behind fluctuations in radiation budget and carbon cycle needed to make accurate projections regarding future temperature rise. At the same time, cooperating with research institutions having a climate numerical model, it contributes to reduction of errors in temperature rise prediction derived from the climate numerical model and improvement of accuracy of prediction of various environmental changes. SGLI mounted on GCOM-C is the succession sensor of the Global Imager (GLI) mounted on ADEOS-II (MIDORI II) and is the Imaging Radiometer which measures the radiation from near-ultraviolet to thermal infrared region (380 nm-12 um) in 19 channels. Global observation of once for approximately every two days is possible at mid-latitude near Japan by observation width at ground greater than 1,000 km. In addition, SGLI realizes high resolution than the similar global sensor and has a polarized observation function and a multi-angle observation function.

https://developers.google.com/earth-engine/datasets/catalog/JAXA_GCOM-C_L3_OCEAN_CHLA_V3#description

In [14]:

def download_chlorophyll(date_ini, date_end, rectangle=RECTANGLE, scale=500):
    
    rectangle = ee.Geometry.Rectangle(rectangle)
    rectangle_bounds = rectangle.bounds().getInfo()['coordinates'][0]
    # dataset elected: https://developers.google.com/earth-engine/datasets/catalog/JAXA_GCOM-C_L3_OCEAN_CHLA_V3
    chlor_a_median = ee.ImageCollection('JAXA/GCOM-C/L3/OCEAN/CHLA/V3')

    chlor_a_median_clipped = chlor_a_median.filterDate(date_ini, date_end).filterBounds(rectangle).median()#.clip(rectangle)


    # Define the scale and maxPixels parameters for reduceRegion()
    scale = scale
    max_pixels = 1e13
    chlor_a_with_latlon = chlor_a_median_clipped.addBands(chlor_a_median_clipped.pixelLonLat())

    # Get the chlorophyll concentration and coordinates as lists
    chlor_a_data = chlor_a_with_latlon.reduceRegion(
        reducer=ee.Reducer.toList(numOptional=1),
        geometry=rectangle,
        scale=scale,
        maxPixels=max_pixels
    )

    # Get the list of latitudes and longitudes
    latitudes = chlor_a_data.get('latitude').getInfo()
    longitudes = chlor_a_data.get('longitude').getInfo()

    # Get the chlorophyll concentration as a list
    chlor_a_list = chlor_a_data.get('CHLA_AVE').getInfo()

    # Check if the lengths of the lists are the same
    assert len(latitudes) == len(longitudes), "Error: The lengths of the lists are not the same."
    assert len(latitudes) == len(chlor_a_list), "Error: The lengths of the lists are not the same."



    df = pd.DataFrame({'chlor': chlor_a_list, 'lat': latitudes, 'lon': longitudes})
    df = df.dropna()

    dict_geometries = {}
    for val in df.chlor.unique():
        geometry = []
        for row in df.query(f'chlor=={val}').index:
            geometry += [(df.loc[row, 'lon'], df.loc[row, 'lat'])]

        dict_geometries[val] = gpd.GeoSeries(MultiPoint(geometry)).convex_hull[0]


    df.to_csv(f'../data/chlorophyll_points_{date_ini}_to_{date_end}.csv', index=False)    
    (pd.DataFrame().from_dict(dict_geometries, orient='index')
                   .reset_index()
                   .rename(columns={'index':'chlor', 0:'geometry'})
                   .to_csv(f'../data/chlorophyll_polygons_{date_ini}_to_{date_end}.csv', index=False))

In [19]:

def download_temperature(date_ini, date_end, rectangle=RECTANGLE, scale=500):
    
    rectangle = ee.Geometry.Rectangle(rectangle)
    rectangle_bounds = rectangle.bounds().getInfo()['coordinates'][0]
    # dataset elected: https://developers.google.com/earth-engine/datasets/catalog/JAXA_GCOM-C_L3_OCEAN_CHLA_V3
    chlor_a_median = ee.ImageCollection('JAXA/GCOM-C/L3/OCEAN/SST/V3')

    chlor_a_median_clipped = chlor_a_median.filterDate(date_ini, date_end).filterBounds(rectangle).median()#.clip(rectangle)


    # Define the scale and maxPixels parameters for reduceRegion()
    scale = scale
    max_pixels = 1e13
    chlor_a_with_latlon = chlor_a_median_clipped.addBands(chlor_a_median_clipped.pixelLonLat())

    # Get the chlorophyll concentration and coordinates as lists
    chlor_a_data = chlor_a_with_latlon.reduceRegion(
        reducer=ee.Reducer.toList(numOptional=1),
        geometry=rectangle,
        scale=scale,
        maxPixels=max_pixels
    )

    # Get the list of latitudes and longitudes
    latitudes = chlor_a_data.get('latitude').getInfo()
    longitudes = chlor_a_data.get('longitude').getInfo()
    # Get the chlorophyll concentration as a list
    temp_list = chlor_a_data.get('SST_AVE').getInfo()

    # Check if the lengths of the lists are the same
    assert len(latitudes) == len(longitudes), "Error: The lengths of the lists are not the same."
    assert len(latitudes) == len(temp_list), "Error: The lengths of the lists are not the same."


    df = pd.DataFrame({'temp': temp_list, 'lat': latitudes, 'lon': longitudes})
    df = df.dropna()

    dict_geometries = {}
    for val in df.temp.unique():
        geometry = []
        for row in df.query(f'temp=={val}').index:
            geometry += [(df.loc[row, 'lon'], df.loc[row, 'lat'])]

        dict_geometries[val] = gpd.GeoSeries(MultiPoint(geometry)).convex_hull[0]


    df.to_csv(f'../data/temperature_points_{date_ini}_to_{date_end}.csv', index=False)    
    (pd.DataFrame().from_dict(dict_geometries, orient='index')
                   .reset_index()
                   .rename(columns={'index':'temp', 0:'geometry'})
                   .to_csv(f'../data/temperature_polygons_{date_ini}_to_{date_end}.csv', index=False))

In [20]:
for i in range(len(DATES)-1):
    download_chlorophyll(DATES[i], DATES[i+1])
    download_temperature(DATES[i], DATES[i+1])

---

In [8]:

# # Define the region of interest
# roi = ee.Geometry.Rectangle([-73.8, -40.7, -73.0, -39.8])

# # Define the date range
# start_date = ee.Date('2021-01-01')
# end_date = ee.Date('2021-12-31')

# # Define the chlorophyll dataset
# chlor_a = ee.ImageCollection('NASA/OCEANDATA/MODIS-Aqua/L3SMI') \
#              .filterDate(start_date, end_date) \
#              .select('chlor_a')

# # Filter the dataset by the region of interest
# chlor_a = chlor_a.filterBounds(roi)

# # Reduce the dataset to a single image by taking the median
# chlor_a_median = chlor_a.median()

# # Convert the chlorophyll concentration image to a format that can be displayed on the map
# vis_params = {'min': 0, 'max': 20, 'palette': 'blue,green,yellow,orange,red'}
# chlor_a_vis = chlor_a_median.visualize(**vis_params)

# # Create a map centered around Valdivia, Chile
# map = folium.Map(location=[-39.83, -73.24], zoom_start=9)

# # Add the chlorophyll concentration layer to the map
# folium.TileLayer(
#     tiles=chlor_a_vis.getThumbURL({'region': roi.getInfo(), 'format': 'png'}),
#     attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
#     overlay=True,
#     name='Chlorophyll Concentration',
# ).add_to(map)

# # Add a layer control to the map
# folium.LayerControl().add_to(map)

# # Display the map
# display(map)


In [14]:
import ee
import folium

# Initialize Earth Engine
ee.Initialize()

# Define the region of interest
roi = ee.Geometry.Rectangle([-73.8, -40.7, -73.0, -39.8])

# Define the date range
start_date = ee.Date('2021-01-01')
end_date = ee.Date('2021-12-31')

# Define the chlorophyll dataset
chlor_a = ee.ImageCollection('NASA/OCEANDATA/MODIS-Aqua/L3SMI') \
             .filterDate(start_date, end_date) \
             .select('chlor_a')

# Filter the dataset by the region of interest
chlor_a = chlor_a.filterBounds(roi)

# Reduce the dataset to a single image by taking the median
chlor_a_median = chlor_a.median()

# Clip the chlorophyll concentration image to the region of interest
chlor_a_median_clipped = chlor_a_median.clip(roi)

# Define the visualization parameters
vis_params = {
    'min': 0,
    'max': 5,
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

# Define the map center
center = roi.centroid().coordinates().reverse().getInfo()

# Create a folium map object
my_map = folium.Map(location=center, zoom_start=9, height=500)

# Add the chlorophyll concentration layer to the map
folium_ee_layer = folium.TileLayer(
    tiles=str(chlor_a_median_clipped.getMapId(vis_params)['tile_fetcher'].url_format),
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Chlorophyll Concentration',
    control=True
)
my_map.add_child(folium_ee_layer)

# Add the region of interest to the map
folium.GeoJson(
    roi.getInfo(),
    name='Region of Interest',
    style_function=lambda x: {'color': 'red', 'weight': 2, 'fillOpacity': 0},
    tooltip='ROI'
).add_to(my_map)

# Display the map
display(my_map)
